# RBF Radial Based Function
### Using Tensorflow / Keras

## Imports

In [ ]:
from utilities.std_imports import *
import matplotlib.cm as cm
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, LSTM, RepeatVector, TimeDistributed

In [14]:
#setting
data = 'D:/data/csv/titanic.csv'
kval = 10
itertot = 40
sigma = 1.2
itergd = 300

def transforminput(param, center):
    newinput = np.zeros((len(param), len(center))).astype('float32')
    for i in range(len(param)):
        for j in range(len(center)):
            newinput[i,j] = np.exp(-(np.sum((param[i] - center[j])**2.0)**0.5) / sigma**2.0)
    return newinput

def generatemodel(numparam):
    model = Sequential()
    model.add(Dense(1, input_dim=numparam, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

dataread = np.genfromtxt(data, delimiter=',')[1:,1:]
alldata = []
for i in range(len(dataread)):
    if np.isnan(dataread[i,-2]):
        continue
    alldata.append(dataread[i])
alldata = np.asarray(alldata)

#dividing data
trainparam = alldata[:600,1:]
trainlabel = alldata[:600,0]
testparam = alldata[600:,1:]
testlabel = alldata[600:,0]

#### Normalization

In [15]:
std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))

for i in range(len(trainparam[0])):
    std[i] = np.std(trainparam[:,i])
    rata[i] = np.mean(trainparam[:,i])
    trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
    testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

#### Search k-mean

In [18]:
#init kmean
kmean = np.zeros((kval, len(trainparamnorm[0])))
for i in range(kval):
    for j in range(len(kmean[0])):
        kmean[i,j] = random.uniform(min(trainparamnorm[:,j]),max(trainparamnorm[:,j]))

#looping of real algorithm
distmin = np.zeros((len(trainparamnorm)))
for i in range(itertot):
    print('iterasi ke', i)
    for j in range(len(distmin)):
        #determine euclid distance
        distall = np.sum((trainparamnorm[j] - kmean)**2.0, axis=1)**0.5
        distmin[j] = np.argmin(distall)

#search new k mean
for j in range(kval):
    clust = []
    for k in range(len(distmin)):
        if distmin[k] == j:
            clust.append(trainparamnorm[k])
    if len(clust) > 0:
        kmean[j] = np.mean(np.asarray(clust), axis=0)

#tranform our input
newinput = transforminput(trainparamnorm, kmean)
print(trainlabel)

iterasi ke 0
iterasi ke 1
iterasi ke 2
iterasi ke 3
iterasi ke 4
iterasi ke 5
iterasi ke 6
iterasi ke 7
iterasi ke 8
iterasi ke 9
iterasi ke 10
iterasi ke 11
iterasi ke 12
iterasi ke 13
iterasi ke 14
iterasi ke 15
iterasi ke 16
iterasi ke 17
iterasi ke 18
iterasi ke 19
iterasi ke 20
iterasi ke 21
iterasi ke 22
iterasi ke 23
iterasi ke 24
iterasi ke 25
iterasi ke 26
iterasi ke 27
iterasi ke 28
iterasi ke 29
iterasi ke 30
iterasi ke 31
iterasi ke 32
iterasi ke 33
iterasi ke 34
iterasi ke 35
iterasi ke 36
iterasi ke 37
iterasi ke 38
iterasi ke 39
[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1.

#### Gradient descent session

In [20]:
mod = generatemodel(kval)
mod.fit(newinput, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)

#predict session
#transform test data
newinputtest = transforminput(testparamnorm, kmean)
lifeprob = mod.predict(newinputtest)

#determine performance
#determine biner accuracy
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
    if lifeprob[i] > 0.5:
        binpred[i] = 1.
score = 0
for i in range(len(testlabel)):
    if binpred[i] == testlabel[i]:
        score += 1
accbin = float(score) / float(len(testlabel))

#determine brier score
brierscore = 0
for i in range(len(testlabel)):
    brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))
for i in range(len(testlabel)):
    print(lifeprob[i], testlabel[i])
print(accbin, brierscore[0])

Epoch 1/300
600/600 [==============================] - 0s 652us/step - loss: 0.6861 - acc: 0.6133
Epoch 2/300
600/600 [==============================] - 0s 33us/step - loss: 0.6834 - acc: 0.6117
Epoch 3/300
600/600 [==============================] - 0s 35us/step - loss: 0.6811 - acc: 0.6117
Epoch 4/300
600/600 [==============================] - 0s 35us/step - loss: 0.6789 - acc: 0.6100
Epoch 5/300
600/600 [==============================] - 0s 37us/step - loss: 0.6768 - acc: 0.6083
Epoch 6/300
600/600 [==============================] - 0s 37us/step - loss: 0.6750 - acc: 0.6083
Epoch 7/300
600/600 [==============================] - 0s 33us/step - loss: 0.6733 - acc: 0.6100
Epoch 8/300
600/600 [==============================] - 0s 35us/step - loss: 0.6718 - acc: 0.6100
Epoch 9/300
600/600 [==============================] - 0s 35us/step - loss: 0.6704 - acc: 0.6100
Epoch 10/300
600/600 [==============================] - 0s 35us/step - loss: 0.6689 - acc: 0.6100
Epoch 11/300
600/600 [======

## Credits & Links

https://medium.com/datadriveninvestor/building-radial-basis-function-network-with-keras-estimating-survivors-of-titanic-a06c2359c5d9